### xGPR for active learning on the fluorescence dataset

We here compare xGPR with Hie et al. 2020 and show that an approximate GP achieves the same results
as an exact GP on this dataset. The code for data preprocessing has been borrowed from
https://github.com/brianhie/uncertainty/.

In [1]:
import os

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

import xGPR
from xGPR.xGP_Regression import xGPRegression
from xGPR.data_handling.dataset_builder import build_online_dataset

import scipy.stats as ss

In [2]:
if "auxiliary_experiments" in os.getcwd():
    os.chdir(os.path.join("..", "benchmark_evals", "sarkisyan_et_al", "data",
                         "sarkisyan2016gfp"))

In [3]:
#This code is all borrowed from Hie et al., to ensure we preprocess the data in the same way.

def acquisition_rank(y_pred, var_pred, beta=1.):
    return ss.rankdata(y_pred) + (beta * ss.rankdata(-var_pred))

def load_embeddings(fname):
    X, meta = [], []
    with open(fname) as f:
        for line in f:
            if line.startswith('>'):
                mutations, n_mut, brightness = line[1:].split('_')
                n_mut = int(n_mut)
                brightness = float(brightness)
                X.append([ float(x) for x in f.readline().split() ])
                meta.append([ mutations, n_mut, brightness ])
    X = np.array(X)
    meta = pd.DataFrame(meta, columns=[
        'mutations', 'n_mut', 'brightness',
    ])
    return X, meta

def split_X(X, meta):
    X_train, X_test, y_train, y_test = [], [], [], []
    mutations_test = []
    for i in range(X.shape[0]):
        n_mut = meta.n_mut[i]
        if n_mut > 1:
            X_test.append(X[i])
            y_test.append(meta.brightness[i])
            mutations_test.append(meta.mutations[i])
        else:
            X_train.append(X[i])
            y_train.append(meta.brightness[i])
    return (np.array(X_train), np.array(y_train),
            np.array(X_test), np.array(y_test),
            mutations_test)



In [4]:
def prep_data():
    X, meta = load_embeddings(
        'embeddings_.txt'
    )
    X_train, y_train, X_test, y_test, mutations_test = split_X(
        X, meta
    )
    #Interestingly, Hie et al. rearrange the range of the y-values,
    #which actually improves performance. This does not seem to
    #have been done by most authors in experiments on the fluorescence
    #dataset (e.g. the TAPE benchmark). Note that because of this
    #rearrangement, we need to add 3 back to predicted y-values at the end
    #of the process.
    y_train -= 3.
    y_test -= 3.
    y_train[y_train < 0.] = 0.
    y_test[y_test < 0.] = 0.
    
    tdset = build_online_dataset(X_train, y_train, chunk_size=2000, normalize_y = False)
    return tdset, X_train, X_test, y_train, y_test

In [5]:
tdset, X_train, X_test, y_train, y_test = prep_data()

In [6]:
#It is also possible to use 1024 variance rffs instead of 2048,
#or to use 4096 fitting rffs instead of 8192; performance drops
#but only very slightly. training_rffs here is irrelevant
#since we use preset hyperparameters instead of tuning.
mod = xGPRegression(training_rffs = 12, fitting_rffs = 8192, device = "gpu",
                    variance_rffs = 2048,
                   kernel_choice = "RBF", verbose = True)

#Like Hie et al. 2020, we use default hyperparameters. We use the same
#lengthscale. Hie et al. used lambda**2=0; xGPR won't allow you to do
#that for numerical stability reasons, but (1e-6)**2 is pretty
#close!
preset_hparams = np.log(np.array([1e-6, 1, 1]))
preconditioner, ratio = mod.build_preconditioner(tdset,
                                         max_rank = 512, method = 'srht',
                                                preset_hyperparams = preset_hparams)
mod.fit(tdset, preconditioner = preconditioner, 
        mode = "cg", tol=1e-7, preset_hyperparams = preset_hparams)

Chunk 0 complete.
starting fitting
Iteration 0
Iteration 5
Iteration 10
Iteration 15
Iteration 20
Iteration 25
Iteration 30
Iteration 35
Iteration 40
Iteration 45
Iteration 50
Now performing variance calculations...
Fitting complete.


In [7]:
#acquisition_rank indicates which test datapoints would be experimentally
#evaluated first using a typical Bayesian optimization paradigm
preds, var = mod.predict(X_test, chunk_size=2000)
rankings = acquisition_rank(preds, var)

acq_argsort = np.argsort(-rankings)
top_fifty = acq_argsort[:50]
top_500 = acq_argsort[:500]

In [8]:
#Surprisingly, our approximate GP outperforms the exact GP in Hie et al. on this
#metric. They observed a correlation of 0.78 between acquisition rank and
#actual fitness, xGPR is here slightly better.
ss.spearmanr(rankings, y_test)

SpearmanrResult(correlation=0.801106301644017, pvalue=0.0)

In [9]:
#This result is ~ identical to the one in Hie et al.
print(np.mean(y_test[top_fifty]) + 3)
print(np.mean(y_test[top_500]) + 3)

3.749814175677
3.7211014590436
